# SLQALCHEMY - PLAIN & SEMI PLAIN SQL

In [131]:
from sqlalchemy import create_engine, Table, MetaData, Column, Integer, String, select, func, text, ForeignKey

## CONNECTION

In [132]:
engine = create_engine("sqlite:///db/heroes.db")

In [133]:
connexion = engine.connect()

## QUERY

In [134]:
query = text("""select * from heroes limit 10""")

In [135]:
connexion.execute(query).fetchall()

[(1, 'Crareek'),
 (2, 'Tan'),
 (3, 'Huzzt'),
 (4, 'Pamble'),
 (5, 'Grimm'),
 (6, 'Jarboyd'),
 (7, 'Cyne'),
 (8, 'Wendner'),
 (9, 'Morbid'),
 (10, 'Grrdy')]

## METADATA

In [136]:
metadata = MetaData(bind=connexion)

> METADATA is used to store database object info/data, such as schema

## TABLE

- To get Column arguments such as nullable, unique, primary_key using sqlite vscode extension, connect to the db, and in sqlite explorer, hover the column to get them

- Otherwise use sql browser

In [137]:
heroes_table = Table('heroes', metadata, 
                     Column('hero_id', Integer, nullable=False, primary_key=True),
                     Column('name', String(20), unique=True, nullable=False),
                     extend_existing= True)

## WHERE , ORDER BY

In [138]:
query = (select([heroes_table])
            .where(heroes_table.c.name.startswith("C"))
            .order_by(heroes_table.c.name.asc()))
connexion.execute(query).fetchall()

[(109, 'Chuz'),
 (42, 'Cinder'),
 (56, 'Cookerry'),
 (71, 'Cookie'),
 (38, 'Crane'),
 (1, 'Crareek'),
 (69, 'Crystal'),
 (7, 'Cyne')]

## AND, OR operator

In [139]:
query = (select([heroes_table])
            .where(heroes_table.c.name.startswith("C") & (heroes_table.c.hero_id < 50) )
            .order_by(heroes_table.c.hero_id))
connexion.execute(query).fetchall()

[(1, 'Crareek'), (7, 'Cyne'), (38, 'Crane'), (42, 'Cinder')]

In [140]:
query = (select([heroes_table])
            .where(heroes_table.c.name.startswith("C") | (heroes_table.c.name.startswith("P")) )
            .order_by(heroes_table.c.name))
connexion.execute(query).fetchall()

[(109, 'Chuz'),
 (42, 'Cinder'),
 (56, 'Cookerry'),
 (71, 'Cookie'),
 (38, 'Crane'),
 (1, 'Crareek'),
 (69, 'Crystal'),
 (7, 'Cyne'),
 (70, 'Pador'),
 (4, 'Pamble'),
 (28, 'Perabiel'),
 (32, 'Pierceslight'),
 (41, 'Pontihill')]

## INSERT

### INSERT SINGLE VALUE USING PLAIN TEXT

> sqlachemy request text for plain text query, as deprecation notice effective in coming 2.0 release

In [141]:
value = {"name": "Toto"}
query = text("insert into heroes (name) values (:name)")

In [142]:
connexion.execute(query, value)

In [143]:
query = text("select * from heroes where name = (:name)")
connexion.execute(query, value).fetchall()

[(123, 'Toto')]

### INSERT MULTIPLE VALUES USING PLAIN TEXT QUERY

In [144]:
values = [{"name": "Titi"}, {"name": "Nemo"}]
query = text("insert into heroes (name) values (:name)")

In [145]:
connexion.execute(query, values)

#### VERIFY INSERTED VALUES

In [146]:
txt_in = ', '.join(["'{0}'".format(v['name']) for v in [*values, value]])
txt = f"select * from heroes where name in ({txt_in})"
print(txt)

select * from heroes where name in ('Titi', 'Nemo', 'Toto')


> NB: it is important using plain text query to surround values with quotes

In [147]:
verify_query = text(txt)

In [148]:
connexion.execute(verify_query).fetchall()

[(125, 'Nemo'), (124, 'Titi'), (123, 'Toto')]

## DELETE

### DELETE MULTIPLE VALUES USING PLAIN TEXT QUERY

In [149]:
values = [{"name": "Titi"}, {"name": "Toto"}, {"name": "Nemo"}]
query = text("delete from heroes where name = (:name)")

In [150]:
connexion.execute(query, values)

#### VERIFY DELETED DATA

In [151]:
verify_query = text("select * from heroes where name = (:name)")
assert not connexion.execute(verify_query, values).fetchall()

## AGGREGATE DATA

### Loop over results

In [152]:
query = """select * from heroes limit 10"""
results = connexion.execute(query)

In [153]:
for row in results:
    print(row)

(1, 'Crareek')
(2, 'Tan')
(3, 'Huzzt')
(4, 'Pamble')
(5, 'Grimm')
(6, 'Jarboyd')
(7, 'Cyne')
(8, 'Wendner')
(9, 'Morbid')
(10, 'Grrdy')


In [154]:
battle_events_table = Table('battle_events', metadata,
                            Column('battle_event_id', Integer, primary_key=True, nullable=False),
                            Column('battle_participant_id', Integer, nullable=False),
                            Column('battle_event_type_id', Integer, nullable=False),
                            Column('rubies_gained', Integer),
                            Column('timestamp', Integer, nullable=False)
                            )

### FUNC

#### FUNC SUM

In [155]:
query = select([func.sum(battle_events_table.c.rubies_gained)])
result = connexion.execute(query).scalar()
result

72847

#### FUNC MAX

In [156]:
query = select([func.max(battle_events_table.c.rubies_gained)])
result = connexion.execute(query)
result.scalar()

15

### JOINING TABLES

In [157]:
metadata.remove(battle_events_table)

> NB: since battle_events table was not properlly definied, redo to include ForeignKey

#### Foreign Key

In [158]:
battle_events_table = Table("battle_events", metadata,
                            Column("battle_event_id", Integer, nullable=False, primary_key=True),
                            Column("battle_participant_id", Integer, ForeignKey("battle_participants.battle.battle_participant_id"), nullable=False),
                            Column("battle_event_type_id", Integer, ForeignKey("battle_event_type.battle_event_type_id"), nullable=False),
                            Column("rubies_gained", Integer),
                            Column("timestamp", Integer, nullable=False))

In [159]:
battle_participants_table = Table("battle_participants", metadata,
                                  Column("battle_participant_id", Integer, nullable=False, primary_key=True),
                                  Column("user_id", Integer, ForeignKey("users.user_id"), nullable=False),
                                  Column("battle_id", Integer, ForeignKey("battles.battle_id"), nullable=False),
                                  Column("hero_id", Integer, ForeignKey("heroes.hero_id"), nullable=False))

> Joining data from `heroes`, `battle_events` and `battle_participants` tables

In [160]:
join = (battle_participants_table
    .join(battle_events_table,
          battle_participants_table.c.battle_participant_id == battle_events_table.c.battle_participant_id)
    .join(heroes_table,
          battle_participants_table.c.hero_id == heroes_table.c.hero_id))

> Query this join to select what hero had the most event in 

In [161]:
query = (select([heroes_table.c.name,
                func.count(battle_events_table.c.battle_event_id).label('total_events')])
         .select_from(join)
         .group_by(heroes_table.c.name)
         .order_by(Column("total_events").desc()))

In [162]:
results = connexion.execute(query)
results.fetchall()

[('Crareek', 3687),
 ('Pamble', 2054),
 ('Huzzt', 2036),
 ('Tan', 1946),
 ('Grimm', 1820),
 ('Jade', 1780),
 ('Hillstomp', 1736),
 ('Mako', 1719),
 ('Grrdy', 1528)]

> From heroes Jade, Grimm, Mako, which of the three had the most hero kills

- hero kill is a battle_events.battle_event_type_id of value 1 and name 'HERO_KILL'
- the kills are represented by the entries in battle_events, which holds reference to the battle_event_type, and the hero_id which did won the battle

In [163]:
battle_event_types_table = Table("battle_event_types", metadata,
                                Column("battle_event_type_id", Integer, nullable=False, primary_key=True),
                                Column("name", String, nullable=False, unique=True))

- Solution with reuse :

In [164]:
join_bet_be_bp_h_1 = (battle_participants_table
    .join(battle_events_table,
          battle_participants_table.c.battle_participant_id == battle_events_table.c.battle_participant_id)
    .join(heroes_table,
          battle_participants_table.c.hero_id == heroes_table.c.hero_id)
    .join(battle_event_types_table,
          battle_event_types_table.c.battle_event_type_id == battle_events_table.c.battle_event_type_id))

In [165]:
q = (select([heroes_table.c.name,
            func.count(battle_events_table.c.battle_event_id).label('total_kills')])
        .select_from(join_bet_be_bp_h_1)
        .group_by(heroes_table.c.name)
        .where(battle_event_types_table.c.name == "HERO_KILL")
        .where((heroes_table.c.name == "Jade") | (heroes_table.c.name == "Grimm") | (heroes_table.c.name == "Mako"))
        .order_by(Column("total_kills").desc())
     )

In [166]:
connexion.execute(q).fetchall()

[('Grimm', 700), ('Jade', 372), ('Mako', 338)]

- Same solution with plain text sql query :

In [167]:
join_bet_be_bp_h = """select h.name, count(be.battle_event_id)
    from heroes as h
    inner join battle_participants as bp
    on h.hero_id = bp.hero_id
    inner join battle_events as be
    on bp.battle_participant_id = be.battle_participant_id
    inner join battle_event_types as bet
    on be.battle_event_type_id = bet.battle_event_type_id
    where bet.name = 'HERO_KILL' and (h.name = 'Grimm' or h.name = 'Jade' or h.name = 'Mako')
    group by h.name
    order by count(be.battle_event_id) desc
    limit 1
    """

In [168]:
connexion.execute(join_bet_be_bp_h).fetchall()

[('Grimm', 700)]